In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Import Modules
import pandas as pd
import csv
import mysql.connector
from getpass import getpass
from mysql.connector import connect, Error

In [5]:
#Mysql Connect
try:
    # Step 1 — Connect to MySQL server (no DB yet)
    server_connection = connect(
        host="localhost",
        user="root",
        password="Anjana@2025"
    )

    cursor = server_connection.cursor()

    # Step 2 — Create database if it doesn't exist
    cursor.execute("CREATE DATABASE IF NOT EXISTS Expense_Tracker;")
    print("Database 'Expense_Tracker' created or already exists ✔")

    cursor.close()
    server_connection.close()

    # Step 3 — Now connect to the Expense_Tracker database
    with connect(
        host="localhost",
        user="root",
        password="Anjana@2025",
        database="Expense_Tracker"
    ) as connection:
        print("Connected to Expense_Tracker ✔")
        print(connection)

except Error as e:
    print(e)

Database 'Expense_Tracker' created or already exists ✔
Connected to Expense_Tracker ✔


In [6]:
connection.reconnect()

In [7]:
df = pd.read_csv ("Per_Exp_Track.csv")
print(df.head())
print(f"Total rows in CSV: {len(df)}")

         Date   Categories  Paymentmode      Description   Amount  Cashback
0  2024-01-26         Fees         NEFT           Semfee   696.44      5.96
1  2024-01-23  Subcription         Cash          Hotstar   210.29      3.88
2  2024-01-30    Groceries         Cash  Weekly purchase    91.81      0.06
3  2024-01-22         Fees  Credit Card          Termfee   871.12      6.67
4  2024-01-31  Investments         Cash       Postoffice  1395.41      7.66
Total rows in CSV: 1200


In [8]:
#Create Database
create_db_query = "CREATE DATABASE IF NOT EXISTS Expense_Tracker"

with connection.cursor() as cursor:
    cursor.execute(create_db_query)

In [13]:
# Create table
create_table_query = """
CREATE TABLE IF NOT EXISTS per_exp_track(
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Date DATE NOT NULL,
    Categories VARCHAR(300),
    Paymentmode VARCHAR(100),
    Description TEXT,
    Amount DECIMAL(10,2) NOT NULL,
    Cashback DECIMAL(10,2) NOT NULL
);
"""

try:
    with connection.cursor() as cursor:
        cursor.execute(create_table_query)   #  remove multi=True
        connection.commit()
        print("Table 'per_exp_track' created successfully ✔")

except Error as e:
    print(e)


Table 'per_exp_track' created successfully ✔


In [14]:
#Data insert into Mysql table

insert_query = """
INSERT INTO per_exp_track(Date, Categories, Paymentmode, Description, Amount, Cashback)
VALUES(%s, %s, %s, %s, %s, %s)
"""
#Insert Query
batch_size = 1200
data_to_insert = []
for i, row in df.iterrows():
    data_to_insert.append((row['Date'], row['Categories'], row['Paymentmode'], row['Description'], row['Amount'], row['Cashback'],))

with connection.cursor() as cursor:
    cursor.executemany(insert_query, data_to_insert)
    connection.commit()

print (f"{cursor.rowcount} CSV data succesfully inserted Mysql Databases")

1200 CSV data succesfully inserted Mysql Databases


In [15]:
connection.reconnect

<bound method MySQLConnectionAbstract.reconnect of <mysql.connector.connection_cext.CMySQLConnection object at 0x00000242904A0A50>>

In [16]:
#Delete the Duplicate Entries
delete_dup_datas = """
SELECT Date, Categories, Paymentmode, Description, Amount, Cashback, COUNT(*) AS DuplicateCount
FROM per_exp_track
GROUP BY Date, Categories, Paymentmode, Description, Amount, Cashback
HAVING COUNT(*) > 1
"""

with connection.cursor() as cursor:
    cursor.execute(delete_dup_datas)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

In [17]:
#Total Expense for the year
total_exp = """
SELECT SUM(Amount) AS TotalExpense FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(total_exp)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(Decimal('1234863.26'),)


In [18]:
#Total Cashback
cb_query = """
SELECT SUM(Cashback) AS TotalCashback FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(cb_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(Decimal('5936.70'),)


In [19]:
#Avg Amount Query
avg_query = """
SELECT AVG(Amount) AS Avgamount FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(avg_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(Decimal('1029.052717'),)


In [20]:
#Total Expenses for 12 Months
Total_exp_month = """
SELECT DATE_FORMAT(Date, '%Y-%m') AS Month, SUM(Amount) AS Total_Expense 
FROM per_exp_track
WHERE Date >= DATE_SUB(CURDATE(), INTERVAL 12 MONTH) 
GROUP BY DATE_FORMAT(Date, '%Y-%m') 
ORDER BY Month ASC
"""

with connection.cursor() as cursor:
    cursor.execute(Total_exp_month)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('2024-12', Decimal('92378.87'))


In [21]:
#>Amount Rs.1000/-
great_amount_query = """
SELECT * FROM per_exp_track
WHERE Amount > 1000
"""

with connection.cursor() as cursor:
    cursor.execute(great_amount_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(5, datetime.date(2024, 1, 31), 'Investments', 'Cash', 'Postoffice', Decimal('1395.41'), Decimal('7.66'))
(6, datetime.date(2024, 1, 27), 'Vegetables', 'NEFT', 'Weekly purchase', Decimal('1315.89'), Decimal('4.55'))
(7, datetime.date(2024, 1, 24), 'Fees', 'Credit Card', 'Coursefee', Decimal('1084.92'), Decimal('3.01'))
(9, datetime.date(2024, 1, 10), 'Bills', 'UPI', 'Cable', Decimal('1854.07'), Decimal('8.13'))
(11, datetime.date(2024, 1, 8), 'Vegetables', 'NEFT', 'Weekly purchase', Decimal('1250.71'), Decimal('6.32'))
(13, datetime.date(2024, 1, 5), 'Vegetables', 'NEFT', 'Weekly purchase', Decimal('1679.02'), Decimal('0.30'))
(18, datetime.date(2024, 1, 20), 'Fees', 'Cash', 'Semfee', Decimal('1756.40'), Decimal('9.23'))
(19, datetime.date(2024, 1, 27), 'Fees', 'UPI', 'Semfee', Decimal('1574.58'), Decimal('6.31'))
(21, datetime.date(2024, 1, 5), 'Subcription', 'UPI', 'Hotstar', Decimal('1969.79'), Decimal('3.38'))
(23, datetime.date(2024, 1, 2), 'Vegetables', 'UPI', 'Weekly purchase', 

In [22]:
#Amount Desc Order
Desc_Amt_query = """
SELECT * FROM per_exp_track
ORDER BY Amount DESC 
LIMIT 20
"""

with connection.cursor() as cursor:
    cursor.execute(Desc_Amt_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(1075, datetime.date(2024, 11, 1), 'Bills', 'UPI', 'Mobile', Decimal('1999.45'), Decimal('5.86'))
(487, datetime.date(2024, 5, 4), 'Health', 'UPI', 'Bought Medicine', Decimal('1996.76'), Decimal('2.92'))
(1103, datetime.date(2024, 12, 10), 'Investments', 'NEFT', 'Insurance', Decimal('1996.13'), Decimal('0.49'))
(414, datetime.date(2024, 5, 6), 'Subcription', 'Cash', 'Hotstar', Decimal('1992.95'), Decimal('9.73'))
(861, datetime.date(2024, 9, 1), 'Fees', 'UPI', 'Coursefee', Decimal('1991.58'), Decimal('1.53'))
(1016, datetime.date(2024, 11, 20), 'Groceries', 'UPI', 'Weekly purchase', Decimal('1991.25'), Decimal('7.94'))
(779, datetime.date(2024, 8, 29), 'Groceries', 'NEFT', 'Weekly purchase', Decimal('1990.74'), Decimal('3.70'))
(90, datetime.date(2024, 1, 10), 'Subcription', 'NEFT', 'Amazonprime', Decimal('1989.93'), Decimal('8.63'))
(930, datetime.date(2024, 10, 28), 'Fees', 'Cash', 'Semfee', Decimal('1989.89'), Decimal('9.03'))
(1058, datetime.date(2024, 11, 30), 'Bills', 'NEFT', 'Wi

In [23]:
#Categorie Wise Transaction
category_query = """
SELECT Categories, COUNT(*) AS Trancount 
FROM per_exp_track
GROUP BY Categories
ORDER BY Categories ASC
"""

with connection.cursor() as cursor:
    cursor.execute(category_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('Bills', 166)
('Fees', 138)
('Groceries', 143)
('Health', 150)
('Investments', 162)
('Shopping', 142)
('Subcription', 149)
('Vegetables', 150)


In [24]:
#Categorie Wise Transaction
category_query = """
SELECT Categories, COUNT(*) AS Trancount
FROM per_exp_track
GROUP BY Categories
UNION ALL
SELECT 'Total' AS Categories, COUNT(*) AS Trancount
FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(category_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('Fees', 138)
('Subcription', 149)
('Groceries', 143)
('Investments', 162)
('Vegetables', 150)
('Bills', 166)
('Health', 150)
('Shopping', 142)
('Total', 1200)


In [25]:
#Categorie Wise Exp Amount
category_Exp_Amt_query = """
SELECT Categories, SUM(Amount) AS Total, COUNT(*) AS Trancount
FROM per_exp_track
GROUP BY Categories
"""

with connection.cursor() as cursor:
    cursor.execute(category_Exp_Amt_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('Fees', Decimal('152384.97'), 138)
('Subcription', Decimal('150640.59'), 149)
('Groceries', Decimal('138751.97'), 143)
('Investments', Decimal('169181.12'), 162)
('Vegetables', Decimal('146250.66'), 150)
('Bills', Decimal('170284.95'), 166)
('Health', Decimal('157669.82'), 150)
('Shopping', Decimal('149699.18'), 142)


In [26]:
#Paymentmode Wise Transaction
paymode_count_query = """
SELECT Paymentmode, COUNT(*) AS Trancount 
FROM per_exp_track
GROUP BY Paymentmode
UNION ALL
SELECT 'Total' AS Paymentmode, COUNT(*) AS Trancount
FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(paymode_count_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('NEFT', 288)
('Cash', 307)
('Credit Card', 302)
('UPI', 303)
('Total', 1200)


In [27]:
#Group by Month
group_month_query = """
SELECT Categories, MONTH(Date) AS Month, COUNT(*) AS Count, SUM(Amount) AS Totalexp 
FROM per_exp_track
GROUP BY Categories, MONTH(Date)
ORDER BY Categories, Month
"""

with connection.cursor() as cursor:
    cursor.execute(group_month_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('Bills', 1, 16, Decimal('15692.13'))
('Bills', 2, 14, Decimal('13818.00'))
('Bills', 3, 14, Decimal('17499.24'))
('Bills', 4, 14, Decimal('12203.54'))
('Bills', 5, 11, Decimal('10628.67'))
('Bills', 6, 18, Decimal('21645.35'))
('Bills', 7, 14, Decimal('9307.21'))
('Bills', 8, 9, Decimal('11362.57'))
('Bills', 9, 14, Decimal('12858.00'))
('Bills', 10, 16, Decimal('15196.29'))
('Bills', 11, 13, Decimal('16525.42'))
('Bills', 12, 13, Decimal('13548.53'))
('Fees', 1, 15, Decimal('15629.31'))
('Fees', 2, 11, Decimal('15833.45'))
('Fees', 3, 10, Decimal('9614.87'))
('Fees', 4, 13, Decimal('13388.26'))
('Fees', 5, 11, Decimal('10158.25'))
('Fees', 6, 10, Decimal('8334.11'))
('Fees', 7, 8, Decimal('10852.39'))
('Fees', 8, 17, Decimal('17670.58'))
('Fees', 9, 11, Decimal('12618.62'))
('Fees', 10, 9, Decimal('11108.40'))
('Fees', 11, 14, Decimal('15766.91'))
('Fees', 12, 9, Decimal('11409.82'))
('Groceries', 1, 13, Decimal('11144.64'))
('Groceries', 2, 18, Decimal('16224.85'))
('Groceries', 3, 

In [28]:
#May Month Retrieve Data
may_month_query = """
SELECT * FROM per_exp_track
WHERE Date BETWEEN '2024-05-01' AND '2024-05-31'
"""

with connection.cursor() as cursor:
    cursor.execute(may_month_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(401, datetime.date(2024, 5, 27), 'Fees', 'UPI', 'Semfee', Decimal('1647.21'), Decimal('5.05'))
(402, datetime.date(2024, 5, 10), 'Investments', 'NEFT', 'Insurance', Decimal('607.24'), Decimal('7.64'))
(403, datetime.date(2024, 5, 24), 'Shopping', 'Cash', 'Electronic Things', Decimal('1689.72'), Decimal('3.57'))
(404, datetime.date(2024, 5, 15), 'Investments', 'Credit Card', 'Postoffice', Decimal('580.39'), Decimal('3.20'))
(405, datetime.date(2024, 5, 27), 'Groceries', 'NEFT', 'Weekly purchase', Decimal('1215.90'), Decimal('9.83'))
(406, datetime.date(2024, 5, 16), 'Vegetables', 'Cash', 'Weekly purchase', Decimal('1642.10'), Decimal('5.40'))
(407, datetime.date(2024, 5, 14), 'Vegetables', 'Cash', 'Weekly purchase', Decimal('1018.06'), Decimal('2.51'))
(408, datetime.date(2024, 5, 4), 'Bills', 'NEFT', 'Wifi', Decimal('1615.10'), Decimal('7.73'))
(409, datetime.date(2024, 5, 15), 'Health', 'NEFT', 'Health Drinks', Decimal('1486.91'), Decimal('6.11'))
(410, datetime.date(2024, 5, 10), 'S

In [29]:
#Insert Single Data
insert_data_query = """
INSERT INTO per_exp_track (Date, Categories, Paymentmode, Description, Amount, Cashback)
VALUES ('2024-06-12', 'Subcription', 'NEFT', 'Netflix', 639.00, 5.00) 
"""

with connection.cursor() as cursor:
    cursor.execute(insert_data_query) 
    result = cursor.fetchall()
    connection.commit()
    print(f"{cursor.rowcount} row(s) updated successfully.")

1 row(s) updated successfully.


In [30]:
#Update Data
update_data_query = """
UPDATE per_exp_track 
SET Paymentmode = 'UPI'
WHERE Description = 'Netflix' AND Date ='2024-06-12'  
"""

with connection.cursor() as cursor:
    cursor.execute(update_data_query) 
    result = cursor.fetchall()
    connection.commit()
    print(f"{cursor.rowcount} row(s) updated successfully.")

1 row(s) updated successfully.


In [31]:
#Specific Month Retrieve Data
Dec_month_query = """
SELECT * FROM per_exp_track
WHERE MONTH(Date) = 12 AND YEAR(Date) = 2024
LIMIT 10
"""

with connection.cursor() as cursor:
    cursor.execute(Dec_month_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(1101, datetime.date(2024, 12, 24), 'Shopping', 'Cash', 'Purchase New Cloth', Decimal('257.34'), Decimal('6.44'))
(1102, datetime.date(2024, 12, 9), 'Groceries', 'UPI', 'Weekly purchase', Decimal('878.95'), Decimal('3.97'))
(1103, datetime.date(2024, 12, 10), 'Investments', 'NEFT', 'Insurance', Decimal('1996.13'), Decimal('0.49'))
(1104, datetime.date(2024, 12, 23), 'Fees', 'UPI', 'Semfee', Decimal('540.33'), Decimal('5.28'))
(1105, datetime.date(2024, 12, 12), 'Investments', 'NEFT', 'Insurance', Decimal('1319.57'), Decimal('6.16'))
(1106, datetime.date(2024, 12, 5), 'Groceries', 'UPI', 'Weekly purchase', Decimal('1894.08'), Decimal('0.10'))
(1107, datetime.date(2024, 12, 22), 'Groceries', 'Credit Card', 'Weekly purchase', Decimal('1013.54'), Decimal('0.52'))
(1108, datetime.date(2024, 12, 19), 'Fees', 'Cash', 'Termfee', Decimal('1544.30'), Decimal('9.34'))
(1109, datetime.date(2024, 12, 21), 'Shopping', 'NEFT', 'New Gadgets', Decimal('368.68'), Decimal('9.60'))
(1110, datetime.date(20

In [32]:
#Most Recent Exp for the date
Rec_date_query = """
SELECT * FROM per_exp_track
ORDER BY Date DESC
LIMIT 10
"""

with connection.cursor() as cursor:
    cursor.execute(Rec_date_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(1120, datetime.date(2024, 12, 30), 'Vegetables', 'Credit Card', 'Weekly purchase', Decimal('555.74'), Decimal('6.28'))
(1181, datetime.date(2024, 12, 30), 'Shopping', 'Cash', 'Electronic Things', Decimal('1551.93'), Decimal('2.24'))
(1178, datetime.date(2024, 12, 30), 'Investments', 'Credit Card', 'Postoffice', Decimal('272.75'), Decimal('8.67'))
(1124, datetime.date(2024, 12, 30), 'Shopping', 'UPI', 'New Furnitures', Decimal('1465.83'), Decimal('6.09'))
(1192, datetime.date(2024, 12, 29), 'Bills', 'NEFT', 'Houserent', Decimal('1896.55'), Decimal('7.30'))
(1180, datetime.date(2024, 12, 29), 'Health', 'Credit Card', 'Doctor Consultation', Decimal('1134.45'), Decimal('1.00'))
(1137, datetime.date(2024, 12, 29), 'Health', 'Cash', 'Nutrition', Decimal('782.52'), Decimal('4.67'))
(1139, datetime.date(2024, 12, 29), 'Shopping', 'Cash', 'Kitchen Things', Decimal('874.33'), Decimal('2.84'))
(1167, datetime.date(2024, 12, 29), 'Vegetables', 'UPI', 'Weekly purchase', Decimal('727.01'), Decimal(

In [33]:
#Most Recent Exp for the Amount
Rec_exp_query = """
SELECT * FROM per_exp_track
ORDER BY Amount DESC
LIMIT 10
"""

with connection.cursor() as cursor:
    cursor.execute(Rec_exp_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(1075, datetime.date(2024, 11, 1), 'Bills', 'UPI', 'Mobile', Decimal('1999.45'), Decimal('5.86'))
(487, datetime.date(2024, 5, 4), 'Health', 'UPI', 'Bought Medicine', Decimal('1996.76'), Decimal('2.92'))
(1103, datetime.date(2024, 12, 10), 'Investments', 'NEFT', 'Insurance', Decimal('1996.13'), Decimal('0.49'))
(414, datetime.date(2024, 5, 6), 'Subcription', 'Cash', 'Hotstar', Decimal('1992.95'), Decimal('9.73'))
(861, datetime.date(2024, 9, 1), 'Fees', 'UPI', 'Coursefee', Decimal('1991.58'), Decimal('1.53'))
(1016, datetime.date(2024, 11, 20), 'Groceries', 'UPI', 'Weekly purchase', Decimal('1991.25'), Decimal('7.94'))
(779, datetime.date(2024, 8, 29), 'Groceries', 'NEFT', 'Weekly purchase', Decimal('1990.74'), Decimal('3.70'))
(90, datetime.date(2024, 1, 10), 'Subcription', 'NEFT', 'Amazonprime', Decimal('1989.93'), Decimal('8.63'))
(930, datetime.date(2024, 10, 28), 'Fees', 'Cash', 'Semfee', Decimal('1989.89'), Decimal('9.03'))
(1058, datetime.date(2024, 11, 30), 'Bills', 'NEFT', 'Wi

In [34]:
#Concat the Categories and Pamentmode
concat_query = """
SELECT CONCAT(Paymentmode, ' - ', Categories) AS Combined_Column, Amount 
FROM per_exp_track 
LIMIT 10
"""

with connection.cursor() as cursor:
    cursor.execute(concat_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('NEFT - Fees', Decimal('696.44'))
('Cash - Subcription', Decimal('210.29'))
('Cash - Groceries', Decimal('91.81'))
('Credit Card - Fees', Decimal('871.12'))
('Cash - Investments', Decimal('1395.41'))
('NEFT - Vegetables', Decimal('1315.89'))
('Credit Card - Fees', Decimal('1084.92'))
('UPI - Subcription', Decimal('582.04'))
('UPI - Bills', Decimal('1854.07'))
('NEFT - Bills', Decimal('570.03'))


In [35]:
#Shopping Expenses
shop_exp_query = """
SELECT COUNT(*) AS total_expenses, SUM(Amount) 
FROM per_exp_track 
WHERE Categories='shopping'
"""

with connection.cursor() as cursor:
    cursor.execute(shop_exp_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(142, Decimal('149699.18'))
